In [51]:
from tqdm import TqdmWarning
import warnings
warnings.filterwarnings("ignore", category=TqdmWarning)
from tqdm.auto import tqdm  # will not emit the IProgress warning now

In [52]:
from utils import base_configs, deps, tr_va_te_split #, models_def, shap_explain
from utils.helpers import rw_csv_helpers, feature_distr_helpers, logging_helpers

In [53]:
CONFIGS = base_configs.get_base_configs()
# path to your global SHAP file

#SWITCH THE FOLLOWING ACCORDINGLY
CONFIGS["RUN_TS"] = "20251202_200541"

In [54]:
# ==========================================
# Merge per-model *_val_report.csv / *_test_report.csv
# into op/all_val_report.csv and op/all_test_report.csv
# ==========================================
from __future__ import annotations
from typing import Dict, Any, Sequence
from pathlib import Path
import pandas as pd


In [55]:
from typing import Dict, Any, Sequence
from pathlib import Path
import pandas as pd


def merge_all_reports(
    CONFIGS: Dict[str, Any],
    model_prefixes: Sequence[str] = ("lr", "rf", "xgb", "xlstm", "tabnet", "mlp"),
):
    """
    Assumes the following directory layout for a given run:

        CONFIGS['DIR_tr_va_te_metric_shap_SAVE_DIR']/
            lr/lr_val_report.csv,   lr_test_report.csv
            rf/rf_val_report.csv,   rf_test_report.csv
            xgb/xgb_val_report.csv, xgb_test_report.csv
            xlstm/xlstm_val_report.csv, xlstm_test_report.csv
            tabnet/tabnet_val_report.csv, tabnet_test_report.csv
            mlp/mlp_val_report.csv, mlp_test_report.csv

    Creates (in the parent 'op/' directory):

        op/all_val_report.csv
        op/all_test_report.csv
    """
    # 1) Use the directory already specified in CONFIGS
    if "DIR_tr_va_te_metric_shap_SAVE_DIR" not in CONFIGS:
        raise KeyError("CONFIGS['DIR_tr_va_te_metric_shap_SAVE_DIR'] is missing.")

    CONFIGS["DIR_tr_va_te_metric_shap_SAVE_DIR"] = f"op/tr_va_te_metric_shap_" + CONFIGS["RUN_TS"] + "/"
    base_dir = Path(CONFIGS["DIR_tr_va_te_metric_shap_SAVE_DIR"]).resolve()
    if not base_dir.exists():
        raise FileNotFoundError(f"Base directory does not exist: {base_dir}")

    # 2) Parent directory (expected to be 'op/')
    op_dir = base_dir.parent
    op_dir.mkdir(parents=True, exist_ok=True)

    test_frames = []
    val_frames = []

    for prefix in model_prefixes:
        model_dir = base_dir / prefix

        test_path = model_dir / f"{prefix}_test_summary.csv"
        val_path  = model_dir / f"{prefix}_val_summary.csv"

        # ---- test ----
        if test_path.exists():
            df_t = pd.read_csv(test_path)
            if "model" not in df_t.columns:
                df_t.insert(0, "model", prefix)
            else:
                df_t["model"] = df_t["model"].fillna(prefix)
            test_frames.append(df_t)
        else:
            print(f"[WARN] Missing test report CSV: {test_path}")

        # ---- validation ----
        if val_path.exists():
            df_v = pd.read_csv(val_path)
            if "model" not in df_v.columns:
                df_v.insert(0, "model", prefix)
            else:
                df_v["model"] = df_v["model"].fillna(prefix)
            val_frames.append(df_v)
        else:
            print(f"[WARN] Missing val report CSV: {val_path}")

    out_paths = {}

    # ---- write merged files into op/ ----
    if test_frames:
        all_test = pd.concat(test_frames, ignore_index=True)
        out_test = op_dir / "all_test_report.csv"
        all_test.to_csv(out_test, index=False)
        print(f"[SAVE] all_test_report.csv → {out_test}")
        out_paths["test"] = str(out_test)

    if val_frames:
        all_val = pd.concat(val_frames, ignore_index=True)
        out_val = op_dir / "all_val_report.csv"
        all_val.to_csv(out_val, index=False)
        print(f"[SAVE] all_val_report.csv → {out_val}")
        out_paths["val"] = str(out_val)

    return out_paths


In [56]:
merge_result = merge_all_reports(CONFIGS)
print(merge_result)

[SAVE] all_test_report.csv → /home/ppanta/puru_proj/proj_v0/hints6_v0/op/all_test_report.csv
[SAVE] all_val_report.csv → /home/ppanta/puru_proj/proj_v0/hints6_v0/op/all_val_report.csv
{'test': '/home/ppanta/puru_proj/proj_v0/hints6_v0/op/all_test_report.csv', 'val': '/home/ppanta/puru_proj/proj_v0/hints6_v0/op/all_val_report.csv'}
